# 讓我們來計算熱點吧


    流程大概是這樣:
    1.照樣先把文章讀進來  分成tuple存起來
    2.然後每讀到一個點 就去mariaDB加一次那個點的count

    因為懶  每次就直接import可能會用到的東西

In [1]:
#encoding=utf-8
import jieba
import pandas as pd
import csv
import geocoder
import pymysql
import math
import numpy as np

    下面比較詳細的解說我放在runningTemplateFianl裡面  這邊就直接跑了
    

In [2]:
jieba.load_userdict("./dict/Chyubu.txt")
df = pd.read_csv('./content/chyubucontent.csv')
tt=[]
with open('./dict/Chyubu.txt',encoding="UTF-8") as f:
    for i in f:
      tt.append(i.strip())
A=set()
B=[]
for partcontent in df['content']:
    words=jieba.cut(partcontent)
    for word in words:
        for ltt in tt:
            if word == ltt:
                A.add(ltt)
    if len(A)>1:
        B.append(list(A))
    A=set()
    

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HOME\AppData\Local\Temp\jieba.cache
Loading model cost 0.735 seconds.
Prefix dict has been built succesfully.


In [17]:
for inner in B:
    print(inner)

['八戶', '久慈', '津輕', '十和田', '新函館北斗', '青森', '新青森', '八食中心']
['白石城', '仙台', '福島', '山形', '山寺']
['仙台', '宮城', '福島']
['藏王狐狸村', '仙台', '松島', '山寺', '仙台城']
['仙台', '吾妻山', '福島']
['藏王狐狸村', '仙台', '福島', '宮城', '秋保溫泉']
['宮城', '喜多方', '福島', '山形', '藏王狐狸村', '仙台']
['五大堂', '山寺', '銀山溫泉', '藏王纜車', '山形', '藏王高原', '立石寺']
['山寺', '山形', '藏王高原']
['山寺', '青森', '銀山溫泉', '松島', '弘前', '角館', '仙台', '田澤湖']
['函館朝市', '新函館北斗', '青森', '函館山夜景', '秋田', '新青森']
['弘前城', '弘前公園', '津輕', '青森', '角館', '蘋果公園', '秋田', '弘前', '新青森']
['盛岡城跡公園', '岩手']
['五大堂', '宮城', '金色堂', '平泉町', '陸奧', '毛越寺', '松島', '中尊寺', '秋田', '仙台', '岩手']
['八戶', '奧入瀨溪流', '十和田湖']
['弘前城', '五大堂', '弘前公園', '瑞巖寺', '松島', '弘前', '仙台']
['武家屋敷', '角館武家屋敷', '弘前公園', '新函館北斗', '青森', '秋田內陸', '新青森', '松島', '毛越寺', '秋田', '弘前', '中尊寺', '角館', '千秋公園', '久保田城']
['弘前', '仙台', '秋田', '十二湖']
['弘前城', '五所川原', '青森', '上杉神社', '山形', '秋田', '弘前', '仙台', '千秋公園', '久保田城']
['青森', '奧入瀨溪流']
['乙女像', '十和田', '青森', '函館山夜景', '八甲田山', '酸湯溫泉', '觀光物產館', '十和田湖', '奧入瀨溪']
['盛岡城跡公園', '青森', '新函館北斗', '秋田', '岩手']
['青森', '弘前城', '津輕']
['青森', '新青森', '青森

    準備來計算次數了
    

In [3]:
db = pymysql.connect("1.162.65.171","bdse0305","bdse0305","bdse03-travel",charset='utf8' )
cur=db.cursor() 

In [4]:
for tmp in B:
    for inner in tmp:
        sql='SELECT * FROM ginwaydu WHERE place="'+inner+'"'
        if cur.execute(sql)==1:
            sql2 = "UPDATE ginwaydu  SET count = count + 1  WHERE place='"+inner+"'"
            cur.execute(sql2)
            db.commit()
db.close()